#  RAG의 기본 개념 / 문서 전처리 과정의 이해

---

## 학습 목표
- RAG(Retrieval-Augmented Generation)의 기본 개념 이해
- 문서 로딩, 청크 분할, 임베딩, 벡터 저장 과정 실습
- LangChain을 활용한 RAG 파이프라인 구축
- 검색 기반 질의응답 시스템 구현

# 환경 설정 및 준비

In [ ]:
# 필수 라이브러리 설치
# uv add langchain langchain-openai langchain-community beautifulsoup4 langchain-chroma

In [ ]:
from dotenv import load_dotenv
load_dotenv()

---

## 1. RAG 개념 및 아키텍처

### 1.1 RAG란?

**RAG (Retrieval-Augmented Generation)** 는 기존 LLM의 한계를 보완하기 위한 방법론:

- **문제점**: LLM은 훈련 시점의 고정된 데이터에만 의존
- **해결책**: 외부 지식 베이스를 동적으로 검색하여 응답 생성 시 활용
- **장점**: 최신 정보, 도메인 특화 지식, 사실 기반 응답 가능

### 1.2 RAG의 핵심 구성요소

```mermaid
graph LR
    A[사용자 쿼리] --> B[문서 검색<br/>Retrieval]
    B --> C[관련 문서]
    C --> D[컨텍스트 증강<br/>Augmentation]
    A --> D
    D --> E[LLM 생성<br/>Generation]
    E --> F[최종 응답]
```

**1. 검색 (Retrieval) 시스템**
- 임베딩 모델: 텍스트를 벡터로 변환
- 벡터 데이터베이스: 임베딩 벡터 저장 및 인덱싱
- 유사도 검색: 코사인 유사도, 유클리드 거리 등

**2. 증강 (Augmentation)**
- 검색된 문서 전처리 및 포맷팅
- 프롬프트 엔지니어링
- 컨텍스트 길이 관리

**3. 생성 (Generation)**
- LLM을 통한 최종 응답 생성
- 검색된 컨텍스트와 원본 질의 결합

### 1.3 RAG vs 기존 접근법 비교

| 특성 | 기존 LLM | 파인튜닝 | RAG |
|------|----------|----------|-----|
| 최신 정보 | ❌ | ❌ | ✅ |
| 구현 복잡도 | 낮음 | 높음 | 중간 |
| 계산 비용 | 낮음 | 높음 | 중간 |
| 소스 추적 | ❌ | ❌ | ✅ |
| 환각 방지 | ❌ | △ | ✅ |

---

## 2. 최신 LangChain을 활용한 RAG 구현

### Step 1: **Indexing**

1. 문서 수집 및 전처리
2. 문서 청크 분할
3. 임베딩 생성
4. 벡터 저장소 구축

`1. 문서 데이터 로드(Load Data)`

- RAG에 사용할 데이터를 불러오는 단계 (검색에 사용될 지식이나 정보)
- 외부 데이터 소스에서 정보를 수집하고, 필요한 형식으로 변환하여 시스템에 로드

In [ ]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader  # type: ignore

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Document 객체로 변환
docs = loader.load() 

# 결과 확인
print(f"Document 개수: {len(docs)}")
print(f"Document 길이: {len(docs[0].page_content)}")
print(f"Document 내용: {docs[0].page_content[5000:6000]}")

In [ ]:
docs[0]

In [ ]:
docs[0] # Document 객체 확인

In [ ]:
docs[0].metadata # Document의 메타데이터 확인

In [ ]:
docs[0].page_content # Document의 페이지 내용 확인

In [ ]:
print(docs[0].page_content)

`2. 문서 청크 분할(Split Texts)`

- 불러온 데이터를 작은 크기의 단위(chunk)로 분할하는 과정
- 자연어 처리(NLP) 기술을 활용하여 큰 문서를 처리가 쉽도록 문단, 문장 또는 구 단위로 나누는 작업
- 검색 효율성을 높이기 위한 중요한 과정

    1. 청크 크기 선택
        - 너무 작은 청크: 문맥 손실
        - 너무 큰 청크: 관련성 저하

    2. 중복 영역 설정
        - 문맥 유지를 위해 필요
        - 일반적으로 10-20% 권장

- LangChain 기본 Text Splitter 종류

    - **`RecursiveCharacterTextSplitter`**
        - 여러 구분자를 우선순위대로 시도하여 문서를 자연스럽게 분할
        - 기본 구분자 순서: `["\n\n", "\n", " ", ""]`
        - 대부분의 경우에 가장 좋은 성능을 보임

    - **`CharacterTextSplitter`**
        - 단일 구분자로만 분할 (예: `\n\n`)
        - 간단한 경우에 사용


- 설치: pip install langchain_text_splitters 또는 uv add langchain_text_splitters

In [ ]:
# Text Split (Documents -> small chunks: Documents)
from langchain_text_splitters import CharacterTextSplitter  # type: ignore

# 1000자씩 잘라서 200자씩 겹치는 Document로 변환
text_splitter = CharacterTextSplitter(
    separator="\n\n",    # 문단 구분자
    chunk_size=1000,     # 문단 길이
    chunk_overlap=200,   # 겹치는 길이
    length_function=len, # 길이 측정 함수
    is_separator_regex=False,   # separator가 정규식인지 여부
)

splitted_docs = text_splitter.split_documents(docs)

# 결과 확인
print(f"Document 개수: {len(splitted_docs)}")
print("\n\n")

for i, doc in enumerate(splitted_docs):
    print(f"Document {i} 길이: {len(doc.page_content)}")
    print(f"Document {i} 내용: {doc.page_content[:100]}...")
    print("-"*50)

> **참고**: 위 출력에서 `Created a chunk of size 9814, which is longer than the specified 1000` 경고가 나타날 수 있습니다. 

- 이는 `chunk_size`가 엄격한 최대값이 아닌 목표값이기 때문입니다. 
- `separator="\n\n"`로 문단 단위로 분할하기 때문에, 하나의 문단이 chunk_size보다 크면 그대로 유지됩니다. 
- 이는 문맥을 유지하기 위한 의도된 동작입니다.

In [ ]:
# 글자 수 기준으로 엄격하게 분할하기 
from langchain_text_splitters import CharacterTextSplitter  # type: ignore

# 1000자씩 잘라서 Document로 변환
text_splitter = CharacterTextSplitter(
    separator="",        # 문단 구분자
    chunk_size=1000,     # 문단 길이
    length_function=len, # 길이 측정 함수
    is_separator_regex=False,   # separator가 정규식인지 여부
)

equally_splitted_docs = text_splitter.split_documents(docs)

# 결과 확인
print(f"Document 개수: {len(equally_splitted_docs)}")
print("\n\n")

for i, doc in enumerate(equally_splitted_docs):
    print(f"Document {i} 길이: {len(doc.page_content)}")
    print("-"*50)

`3. 문서 임베딩 생성(Document Embeddings)`

- 임베딩 모델을 사용하여 텍스트를 벡터로 변환
- 임베딩을 기반으로 유사성 검색에 사용
- 임베딩 모델 선택
   - 성능과 비용 고려
   - 다국어 지원 여부 확인

In [ ]:
# OpenAI Embeddings - 문장 임베딩

from langchain_openai import OpenAIEmbeddings  # type: ignore

# embedding model 생성
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름을 지정 가능 
)

sample_text = "위키피디아 정책 변경 절차를 알려주세요"
embedding_vector = embedding_model.embed_query(sample_text)
print(f"임베딩 벡터의 차원: {len(embedding_vector)}")
print(f"임베딩 벡터: {embedding_vector[:10]}...")

`4. 벡터 저장소 구축 (Vectorstores)`

- 임베딩 벡터를 벡터저장소에 저장 
- 저장된 임베딩을 기반으로 유사성 검색을 수행하는데 활용 

In [ ]:
splitted_docs[0].page_content

In [ ]:
# Chroma 벡터 저장소에 문서 저장하기
from langchain_chroma import Chroma
vector_store = Chroma(embedding_function=embedding_model)

# Document를 VectorStore에 저장
document_ids = vector_store.add_documents(splitted_docs)

# 결과 확인
print(f"저장된 Document 개수: {len(document_ids)}")

In [ ]:
print(f"저장된 Document ID: {document_ids[:5]}...")

In [ ]:
vector_store._collection.count()

In [ ]:
# VectorStore에 저장된 Document 개수 확인
print(f"VectorStore에 저장된 Document 개수: {vector_store._collection.count()}")

> **참고**: Chroma 실행 시 `Failed to send telemetry event` 경고가 나타날 수 있습니다. 

- 이는 Chroma의 사용 통계 수집 기능과 관련된 것으로, 실제 기능에는 영향을 주지 않습니다. 무시하셔도 됩니다.

### Step 2: **Retrieval and generation**

`5. 검색 및 생성`

- `RunnableParallel`과 `RunnablePassthrough`를 조합하여 체인을 구성


In [ ]:
# 벡터 스토어 문서 검색 - 유사도 기반 검색

search_query = "위키피디아 정책 변경 절차를 알려주세요"

results = vector_store.similarity_search(query=search_query, k=2)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
    print("-"*50)

In [ ]:
# 벡터 스토어 검색기 설정 - 유사도 기반 검색

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

# 검색기로 검색하기
results = retriever.invoke(input=search_query)

# 결과 확인
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
    print("-"*50)

In [ ]:
# RunnableParallel을 사용한 RAG 체인

from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 시스템 프롬프트
system_prompt = (
    "다음 검색된 맥락을 사용하여 사용자의 질문에 답하세요. "
    "답을 모르면 모른다고 하고, 추측하지 마세요. "
    "답변은 한국어로 간결하고 정확하게 작성하세요.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# LLM 모델 설정
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

rag_chain = RunnableParallel({
    "context": retriever | RunnableLambda(lambda docs: "\n\n".join([doc.page_content for doc in docs])),
    "input": RunnablePassthrough()
    }) | prompt | llm | StrOutputParser()

query = "위키피디아 정책 변경 절차를 알려주세요"
response = rag_chain.invoke(query)

# 결과 확인
print(f"\n답변:\n{response}")

# [실습 프로젝트]

### RAG 파이프라인 구축하기

이제 배운 내용을 바탕으로 실제 RAG 파이프라인을 직접 구축해보겠습니다.

**목표**: 뉴스 기사를 로드하고, 청크로 분할하여 벡터 저장소에 저장한 후, 질문에 답변하는 RAG 시스템 구축

**단계별 가이드**:
1. 데이터 준비 → 텍스트 처리 → 임베딩 → 검색 → 평가
2. 각 과제마다 구현해야 할 함수와 결과물 정의
3. 단위 테스트를 통한 검증

In [ ]:
# 1단계: 데이터 준비 - 웹문서 검색을 위해 관련 URL 가져오기
web_urls = [
    "https://n.news.naver.com/mnews/article/029/0002927209",
    "https://n.news.naver.com/mnews/article/092/0002358620",
    "https://n.news.naver.com/mnews/article/008/0005136824",
]

In [ ]:
# 2단계: WebBaseLoader를 사용해 텍스트 로드
"""
힌트:
- WebBaseLoader를 사용하여 web_urls의 문서들을 로드하세요
- loader.load()를 호출하여 Document 객체 리스트를 얻습니다
- 로드된 문서의 개수를 출력하여 확인하세요

기대 출력:
- 로드된 문서 개수: 3개 (URL 개수와 동일)
"""
pass

In [ ]:
# 3단계: CharacterTextSplitter로 문서 분할
"""
힌트:
- CharacterTextSplitter를 import하세요
- chunk_size=500, chunk_overlap=100으로 설정하세요
- separator="\n\n"로 문단 단위 분할
- split_documents() 메서드를 사용하여 문서를 분할하세요
- 분할된 청크의 개수를 출력하세요

기대 출력:
- 분할된 청크 개수: 약 20-40개 (문서 내용에 따라 다를 수 있음)
"""
pass

In [ ]:
# 4단계: 임베딩 및 벡터 저장소 구현
"""
힌트:
- OpenAIEmbeddings를 사용하여 임베딩 모델을 생성하세요
- Chroma 벡터 저장소를 생성하세요
- add_documents()를 사용하여 분할된 문서를 벡터 저장소에 추가하세요
- 저장된 문서 개수를 확인하세요

기대 출력:
- 임베딩 모델 생성 완료
- 벡터 저장소에 저장된 문서 개수: (3단계의 청크 개수와 동일)
"""
pass

In [ ]:
# 5단계: RAG 기반 QA 체인 구현
"""
힌트:
- ChatOpenAI로 LLM 모델을 생성하세요 (model="gpt-4.1-nano")
- ChatPromptTemplate으로 프롬프트 템플릿을 만드세요
- 벡터 저장소에서 as_retriever()로 검색기를 생성하세요
- RunnableParallel 사용하여 RAG 체인을 구성하세요

기대 출력:
- RAG 체인 생성 완료
"""
pass

In [ ]:
# 6단계: QA 체인으로 질문 응답
"""
힌트:
- 뉴스 기사와 관련된 질문을 준비하세요
- rag_chain.invoke({"question": 질문})으로 답변을 얻으세요
- response['answer']로 답변 내용을 확인하세요
- response['context']로 검색된 문서를 확인하세요

기대 출력:
- 질문에 대한 답변이 출력됩니다
- 검색된 관련 문서들의 내용이 포함됩니다

예시 질문:
- "기사의 주요 내용을 요약해주세요"
- "주요 인물은 누구인가요?"
- "어떤 사건에 대한 기사인가요?"
"""
pass
